In [91]:
import os, sys, numpy as np, pickle, random
from sklearn.neighbors import kneighbors_graph

In [92]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer, BertForQuestionAnswering
from tqdm import tqdm
from datasets import load_dataset
from dig import DiscretetizedIntegratedGradients
from attributions import run_dig_explanation
from metrics_QnA import comprehensiveness, sufficiency
import monotonic_paths
from captum.attr._utils.common import _format_input_baseline, _reshape_and_sum, _validate_input, _format_input
from captum.attr import LayerConductance, LayerIntegratedGradients

In [93]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

In [94]:
tokenizer = BertTokenizer.from_pretrained('deepset/bert-base-cased-squad2')
model= BertForQuestionAnswering.from_pretrained('deepset/bert-base-cased-squad2')

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [95]:
device= torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)
model.eval()
model.zero_grad()

cpu


In [96]:

def predict(model, inputs_embeds, attention_mask=None):
    output = model(inputs_embeds=inputs_embeds, attention_mask=attention_mask )
    return output.start_logits, output.end_logits

def nn_forward_func(input_embed, attention_mask = None, position_embed = None, type_embed = None, return_all_logits = False):
    global model
    embeds	= input_embed + position_embed + type_embed
    embeds	= model.bert.embeddings.dropout(model.bert.embeddings.LayerNorm(embeds))
    pred	= predict(model, embeds, attention_mask=attention_mask)[0]
    
    if return_all_logits:
        return pred
    else:
        return pred.max(1).values


def construct_input_ref_pair(question, text, ref_token_id, sep_token_id, cls_token_id):
    question_ids = tokenizer.encode(question, add_special_tokens=False)
    text_ids = tokenizer.encode(text, add_special_tokens=False)

    # construct input token ids
    input_ids = [cls_token_id] + question_ids + [sep_token_id] + text_ids + [sep_token_id]

    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(question_ids) + [sep_token_id] + \
        [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(question_ids)

def construct_input_ref_token_type_pair(input_ids, sep_ind=0):
    seq_len = input_ids.size(1)
    token_type_ids = torch.tensor([[0 if i <= sep_ind else 1 for i in range(seq_len)]], device=device)
    ref_token_type_ids = torch.zeros_like(token_type_ids, device=device)# * -1
    return token_type_ids, ref_token_type_ids

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids
    
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

def construct_whole_bert_embeddings(input_ids, ref_input_ids, \
                                    token_type_ids=None, ref_token_type_ids=None, \
                                    position_ids=None, ref_position_ids=None):
    input_embeddings = model.bert.embeddings(input_ids, token_type_ids=token_type_ids, position_ids=position_ids)
    ref_input_embeddings = model.bert.embeddings(ref_input_ids, token_type_ids=ref_token_type_ids, position_ids=ref_position_ids)
    
    return input_embeddings, ref_input_embeddings

def get_word_embeddings():
	global model
	return model.bert.embeddings.word_embeddings.weight

def construct_word_embedding(model, input_ids):
	return model.bert.embeddings.word_embeddings(input_ids)

def construct_position_embedding(model, position_ids):
	return model.bert.embeddings.position_embeddings(position_ids)

def construct_type_embedding(model, type_ids):
	return model.bert.embeddings.token_type_embeddings(type_ids)

def construct_sub_embedding(model, input_ids, ref_input_ids, position_ids, ref_position_ids, type_ids, ref_type_ids):
	input_embeddings				= construct_word_embedding(model, input_ids)
	ref_input_embeddings			= construct_word_embedding(model, ref_input_ids)
	input_position_embeddings		= construct_position_embedding(model, position_ids)
	ref_input_position_embeddings	= construct_position_embedding(model, ref_position_ids)
	input_type_embeddings			= construct_type_embedding(model, type_ids)
	ref_input_type_embeddings		= construct_type_embedding(model, ref_type_ids)

	return 	(input_embeddings, ref_input_embeddings), \
			(input_position_embeddings, ref_input_position_embeddings), \
			(input_type_embeddings, ref_input_type_embeddings)

def get_base_token_emb(device):
	global model
	return construct_word_embedding(model, torch.tensor([tokenizer.mask_token_id], device=device))

def get_tokens(text_ids):
	global tokenizer
	return tokenizer.convert_ids_to_tokens(text_ids.squeeze())


def get_inputs(question, text, device):
    global model, tokenizer
    ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
    sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
    cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence/
    
    input_ids, ref_input_ids, sep_id = construct_input_ref_pair(question, text, ref_token_id, sep_token_id, cls_token_id)
    token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, sep_id)
    position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
    attention_mask = construct_attention_mask(input_ids)

    (input_embed, ref_input_embed), (position_embed, ref_position_embed), (type_embed, ref_type_embed) = \
            construct_sub_embedding(model, input_ids, ref_input_ids, position_ids, ref_position_ids, token_type_ids, ref_token_type_ids)

    return [input_ids, ref_input_ids, input_embed, ref_input_embed, position_ids, ref_position_ids, position_embed, ref_position_embed, token_type_ids, ref_token_type_ids, type_embed, ref_type_embed, attention_mask]

In [97]:
dataset= load_dataset('squad',split = 'validation',download_mode='force_redownload')
data= list(zip(dataset['question'], dataset['context'] , dataset['answers'], dataset['id']))

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [98]:
upd_data=[]
for question, context, ans, idx in data:
    if len((question+context).split(" ")) < 80:
        upd_data.append((question, context, ans, idx))
len(upd_data)

930

In [99]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

In [100]:
def calculate_attributions(inputs, device, base_token_emb, nn_forward_func, logit_type):
    # computes the attributions for given input

    # move inputs to main device
    inp = [x.to(device) if x is not None else None for x in inputs]

    # compute attribution
    input_ids, ref_input_ids, input_embed, ref_input_embed, position_ids, ref_position_ids, position_embed, ref_position_embed, token_type_ids, ref_token_type_ids, type_embed, ref_type_embed, attention_mask = inp
    
    lig = LayerIntegratedGradients(nn_forward_func, model.bert.embeddings)
    
    if logit_type == 'start':
        attr, deltaa = lig.attribute(inputs=input_embed,
                                          baselines=ref_input_embed,
                                          additional_forward_args=(attention_mask, position_embed, type_embed, False),
                                          return_convergence_delta=True)
   
    else:
        attr, deltaa = lig.attribute(inputs=input_embed, baselines=ref_input_embed,
                                    additional_forward_args=(attention_mask, position_embed, type_embed, False),
                                    return_convergence_delta=True)
    
    
    attributions_sum = summarize_attributions(attr)
    suff, pred	= sufficiency(nn_forward_func,input_ids,input_embed, position_embed, type_embed, attention_mask, base_token_emb, attributions_sum, topk=50)
    comp , pred = comprehensiveness(nn_forward_func, input_ids, input_embed, position_embed, type_embed, attention_mask, base_token_emb, attributions_sum, topk=50)

    return suff, comp, attributions_sum, deltaa

In [101]:
%%time
# get ref token embedding
base_token_emb = get_base_token_emb(device)

# compute the IG attributions for all the inputs
print('Starting attribution computation...')
inputs = []
comps, suffs, deltas, delta_pcs, count = 0, 0, 0, 0, 0
print_step = 1

delta_pcs_list = []
for row in tqdm(upd_data):
    inp = get_inputs(row[0], row[1], device)
    input_ids, ref_input_ids, input_embed, ref_input_embed, position_ids, ref_position_ids, position_embed, ref_position_embed, token_type_ids, ref_token_type_ids, type_embed, ref_type_embed, attention_mask = inp
    
    inputs					= [input_ids, ref_input_ids, input_embed, ref_input_embed, position_ids, ref_position_ids, position_embed, ref_position_embed, token_type_ids, ref_token_type_ids, type_embed, ref_type_embed, attention_mask]
    suff, comp, attrib, delta= calculate_attributions(inputs, device, base_token_emb, nn_forward_func, "start")
    comps		+= comp
    suffs 		+= suff
    deltas+= torch.abs(delta).item()
    count		+= 1

    # print the metrics
    if count % print_step == 0:
        print('Comprehensiveness: ', np.round(comps / count, 4), 'Sufficiency: ', np.round(suffs / count, 4),
               'Avg delta: ', np.round(deltas / count, 4))

print('Comprehensiveness: ', np.round(comps / count, 4), 'Sufficiency: ', np.round(suffs / count, 4), 'Avg delta: ', np.round(deltas / count, 4))


Starting attribution computation...




  0%|          | 0/930 [00:00<?, ?it/s]

  0%|          | 1/930 [00:39<10:07:32, 39.24s/it]

Comprehensiveness:  0.4768 Sufficiency:  0.5059 Avg delta:  0.0629




  0%|          | 2/930 [01:21<10:29:51, 40.72s/it]

Comprehensiveness:  0.7341 Sufficiency:  0.6299 Avg delta:  0.2533


...

Comprehensiveness:  0.7322 Sufficiency:  0.4726 Avg delta:  0.7214




100%|█████████▉| 929/930 [11:10:16<00:58, 58.02s/it]

Comprehensiveness:  0.7323 Sufficiency:  0.4721 Avg delta:  0.721




100%|██████████| 930/930 [11:11:11<00:00, 43.30s/it]

Comprehensiveness:  0.732 Sufficiency:  0.472 Avg delta:  0.7202
Comprehensiveness:  0.732 Sufficiency:  0.472 Avg delta:  0.7202
CPU times: user 10h 32min 20s, sys: 30min 54s, total: 11h 3min 14s
Wall time: 11h 11min 11s
